In [108]:
!mkdir -p fastq_processed/
!mkdir -p fastq_processed/assembled
!mkdir -p fastq_processed/collapsed

## 0. Merge paired-end reads into single read
- https://cme.h-its.org/exelixis/web/software/pear/

In Step 2. I tried using the actual paired-end data ...
    
    %%bash 
    umi_tools extract --bc-pattern=NNNNNNNNN \
    -I \
    fastq_processed/collapsed/ex3_1_S1_R1.fastq \
    --read2-in \
    fastq_processed/collapsed/ex3_1_S1_R2.fastq \
    --log \
    logs/umi_tools_extract/ex3_1.log \
    --stdout \
    fastq_processed/umi_extract/ex3_1_S1_R1.fastq \
    --read2-out \ 
    fastq_processed/umi_extract/ex3_1_S1_R2.fastq
    
then I saw this error 

    ValueError: 
    Read pairs do not match
    M00179:265:000000000-K83HN:1:1107:5417:5996#1 != M00179:265:000000000-K83HN:1:1101:13697:13764#1
    bash: line 11: fastq_processed/umi_extract/ex3_1_S1_R2.fastq: Permission denied


In [ ]:
%%bash
for f in fastq/*_R1*; do 
    f2=${f/_R1/_R2}
    b=`basename $f`
    ass=${b/_S[1-9]*/}
    cm="pear -j 10 -f $f -r $f2 -o fastq_processed/assembled/$ass"
    echo $cm
    $cm
    wait 
done 

In [107]:
ls fastq_processed/assembled/*assembled.fastq

fastq_processed/assembled/ex3_1.assembled.fastq
fastq_processed/assembled/ex3_2.assembled.fastq
fastq_processed/assembled/ex4_1.assembled.fastq
fastq_processed/assembled/ex4_2.assembled.fastq
fastq_processed/assembled/NC1_ex3.assembled.fastq
fastq_processed/assembled/NC1_ex4.assembled.fastq
fastq_processed/assembled/NC2_ex3.assembled.fastq
fastq_processed/assembled/NC2_ex4.assembled.fastq


<!-- Hani shared this chunk of codes/notes. 
 -->

## 1. Collapse identical sequences

### Setup CTK

https://zhanglab.c2b2.columbia.edu/index.php/CTK_usage

https://zhanglab.c2b2.columbia.edu/index.php/CTK_Documentation

#### Installation

My `ctk` conda env for prerequisites 

https://zhanglab.c2b2.columbia.edu/index.php/CTK_Documentation | Download
> czplib (perl): a perl library with various functions for genomic/bioinformatic analysis. Download from github

> CTK (perl): the core algorithm. Download from github


In [11]:
!ls -d ~/Workflows/CTK/*/

/data_gilbert/home/aarab/Workflows/CTK/ctk-1.1.4/
/data_gilbert/home/aarab/Workflows/CTK/czplib-1.0.8/


In [10]:
%%bash
export PERL5LIB=/data_gilbert/home/aarab/Workflows/CTK/czplib-1.0.8
export CTKDIR=/data_gilbert/home/aarab/Workflows/CTK/ctk-1.1.4

echo $CTKDIR

/data_gilbert/home/aarab/Workflows/CTK/ctk-1.1.4


In [109]:
%%bash
export PERL5LIB=/data_gilbert/home/aarab/Workflows/CTK/czplib-1.0.8
export CTKDIR=/data_gilbert/home/aarab/Workflows/CTK/ctk-1.1.4

for f in fastq_processed/assembled/*assembled.fastq; do 
    b=`basename $f`
    echo $f $b; 
    perl $CTKDIR/fastq2collapse.pl -v --tmp-dir tmp $f fastq_processed/collapsed/$b
    wait
done

fastq_processed/assembled/ex3_1.assembled.fastq ex3_1.assembled.fastq
awk '{if(NR%4==1) {print $1}}' fastq_processed/assembled/ex3_1.assembled.fastq > tmp/id
awk '{if(NR%4==2) {print $0}}' fastq_processed/assembled/ex3_1.assembled.fastq > tmp/seq
awk '{if(NR%4==0) {print $0}}' fastq_processed/assembled/ex3_1.assembled.fastq > tmp/qual
paste tmp/id tmp/qual tmp/seq| sort -T tmp -k 3 | uniq -f 2 -c | awk '{print $2"#"$1"\n"$4"\n+\n"$3}' > fastq_processed/collapsed/ex3_1.assembled.fastq
fastq_processed/assembled/ex3_2.assembled.fastq ex3_2.assembled.fastq
awk '{if(NR%4==1) {print $1}}' fastq_processed/assembled/ex3_2.assembled.fastq > tmp/id
awk '{if(NR%4==2) {print $0}}' fastq_processed/assembled/ex3_2.assembled.fastq > tmp/seq
awk '{if(NR%4==0) {print $0}}' fastq_processed/assembled/ex3_2.assembled.fastq > tmp/qual
paste tmp/id tmp/qual tmp/seq| sort -T tmp -k 3 | uniq -f 2 -c | awk '{print $2"#"$1"\n"$4"\n+\n"$3}' > fastq_processed/collapsed/ex3_2.assembled.fastq
fastq_processed/assemb

## 2. Remove the UMIs with umi_tools extract

My `alignment` env  

In [110]:
# !umi_tools

In [111]:
ls fastq_processed/collapsed

ex3_1.assembled.fastq  ex4_2.assembled.fastq    NC2_ex3.assembled.fastq
ex3_2.assembled.fastq  NC1_ex3.assembled.fastq  NC2_ex4.assembled.fastq
ex4_1.assembled.fastq  NC1_ex4.assembled.fastq


In [114]:
%%bash 
mkdir -p fastq_processed/umi_extract;
mkdir -p logs;
mkdir -p logs/umi_tools_extract;
# mkdir -p fastq_processed/trim;
# mkdir -p logs/cutadapt_trim;

fastqDIR='fastq_processed/collapsed'

for fq_file in ${fastqDIR}/*.fastq; do
    fq_base=`basename $fq_file`;
    sample_id=${fq_base/.fastq/};


    echo '--------------' $sample_id '--------------'

    umi_tools extract --bc-pattern=NNNNNNNNN \
     -I $fq_file \
     --log=logs/umi_tools_extract/${sample_id}.log \
     --stdout=fastq_processed/umi_extract/$fq_base \

    #cutadapt -j $JOBS -q 15 -m 20 -U 9 \
    # -o fastq_processed/trim/$fq_base \
    # -p fastq_processed/trim/$fq_base \
    # fastq_processed/umi_extract/$fq1_base fastq_processed/umi_extract/$fq2_base> logs/cutadapt_trim/$log_file;

done


-------------- ex3_1.assembled --------------
-------------- ex3_2.assembled --------------
-------------- ex4_1.assembled --------------
-------------- ex4_2.assembled --------------
-------------- NC1_ex3.assembled --------------
-------------- NC1_ex4.assembled --------------
-------------- NC2_ex3.assembled --------------
-------------- NC2_ex4.assembled --------------


## 3. Align to the specific amplicons the data comes from

In [2]:
!which bwa

/data_gilbert/home/aarab/anaconda3/envs/ctk/bin/bwa


### Align to hg19

#### Genome index
I tried to download `hg19` genome index from [this link](http://ccb.jhu.edu/software/tophat/igenomes.shtml) and it didn't work! 

Instead, I'm downloading `hg19.fa.gz` file from https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/

Then, indexing through
```bash 
bwa index -a bwtsw <genome>.fa
```

In [11]:
ls /data_gilbert/home/aarab/genomes/hg19/

hg19.fa  hg19.fa.amb  hg19.fa.ann  hg19.fa.bwt  hg19.fa.pac  hg19.fa.sa


In [16]:
!mkdir bam

In [17]:
%%bash 
genome='/data_gilbert/home/aarab/genomes/hg19/hg19.fa'
for f in fastq_processed/umi_extract/*; do
    b=`basename $f`
    o=${b/assembled.fastq/bam}
    cm="bwa mem -t 10 -o bam/$o $genome $f"
    echo $cm
    echo `date`
    $cm
done 

bwa mem -t 10 -o bam/ex3_1.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/ex3_1.assembled.fastq
Fri Mar 4 11:41:30 PST 2022
bwa mem -t 10 -o bam/ex3_2.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/ex3_2.assembled.fastq
Fri Mar 4 11:41:45 PST 2022
bwa mem -t 10 -o bam/ex4_1.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/ex4_1.assembled.fastq
Fri Mar 4 11:41:59 PST 2022
bwa mem -t 10 -o bam/ex4_2.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/ex4_2.assembled.fastq
Fri Mar 4 11:42:17 PST 2022
bwa mem -t 10 -o bam/NC1_ex3.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/NC1_ex3.assembled.fastq
Fri Mar 4 11:42:35 PST 2022
bwa mem -t 10 -o bam/NC1_ex4.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/NC1_ex4.assembled.fastq
Fri Mar 4 11:42:50 PST 2022
bwa mem -t 10 -o bam/NC2_ex3.bam /data_gilbert/home/aarab/geno

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 646461 sequences (90163978 bp)...
[M::mem_process_seqs] Processed 646461 reads in 85.552 CPU sec, 8.720 real sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa mem -t 10 -o bam/ex3_1.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/ex3_1.assembled.fastq
[main] Real time: 15.012 sec; CPU: 91.497 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 613290 sequences (85477075 bp)...
[M::mem_process_seqs] Processed 613290 reads in 80.421 CPU sec, 8.200 real sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa mem -t 10 -o bam/ex3_2.bam /data_gilbert/home/aarab/genomes/hg19/hg19.fa fastq_processed/umi_extract/ex3_2.assembled.fastq
[main] Real time: 13.792 sec; CPU: 85.713 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 437494 sequences (100000266 bp)...
[M::process] read 256785 sequences (58707800 bp)...
[M::mem_process_seqs] Processed 437494 reads in 73.8

In [28]:
%%bash 
for bam in bam/*; do
    srt=${bam/.bam/.srt.bam}
    cm="samtools sort -@ 20 -o $srt $bam"
    echo $cm
    echo `date`
    $cm
done 

samtools sort -@ 20 -o bam/ex3_1.srt.bam bam/ex3_1.bam
Fri Mar 4 12:03:05 PST 2022
samtools sort -@ 20 -o bam/ex3_2.srt.bam bam/ex3_2.bam
Fri Mar 4 12:03:06 PST 2022
samtools sort -@ 20 -o bam/ex4_1.srt.bam bam/ex4_1.bam
Fri Mar 4 12:03:08 PST 2022
samtools sort -@ 20 -o bam/ex4_2.srt.bam bam/ex4_2.bam
Fri Mar 4 12:03:10 PST 2022
samtools sort -@ 20 -o bam/NC1_ex3.srt.bam bam/NC1_ex3.bam
Fri Mar 4 12:03:11 PST 2022
samtools sort -@ 20 -o bam/NC1_ex4.srt.bam bam/NC1_ex4.bam
Fri Mar 4 12:03:13 PST 2022
samtools sort -@ 20 -o bam/NC2_ex3.srt.bam bam/NC2_ex3.bam
Fri Mar 4 12:03:14 PST 2022
samtools sort -@ 20 -o bam/NC2_ex4.srt.bam bam/NC2_ex4.bam
Fri Mar 4 12:03:15 PST 2022


[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...
[bam_sort_core] merging from 0 files and 20 in-memory blocks...


### Align to amplicons

Let's just align to amplicons!

In [121]:
def write_fasta(path, fa):
    file = open(path, 'w')
    for f in fa:
        file.write('>' + f + '\n')
        file.write(fa[f].upper() + '\n')
    file.close()

In [122]:
fasta = {
    'Exon3_amplicon':
    'GATTCCTACAGGAAGCAAGTAGtaattgatggagaaacctgtctcttggatattctcgacacagcaggtcaagaggagtacagtgcaatgagggaccagtacatgaggactggggagGGCTTTCTTTGTGTATTTGCC'
    ,
    'Exon4_amplicon':
    'GTTGTGGACAGGTTTTGAAAGatatttgtgttactaatgactgtgctataacttttttttctttcccagagaacaaattaaaagagttaaggactctgaagatgtacctatggtcctagtaggaaataaatgtgatttgccttctagaacagtagacacaaaacaggctcaggacttagcaagaagttatggaattccttttattgaaaCATCAGCAAAGACAAGACAG'
}

In [123]:
!mkdir -p sequence

In [125]:
write_fasta('sequence/amplicons.fa',fasta)

In [128]:
!bwa index sequence/amplicons.fa

[bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa index sequence/amplicons.fa
[main] Real time: 0.011 sec; CPU: 0.139 sec


In [129]:
ls sequence

amplicons.fa      amplicons.fa.ann  amplicons.fa.pac
amplicons.fa.amb  amplicons.fa.bwt  amplicons.fa.sa


In [148]:
!mkdir bam

In [149]:
%%bash 
for f in fastq_processed/umi_extract/*; do
    b=`basename $f`
    o=${b/assembled.fastq/bam}
    cm="bwa mem -t 10 -o bam/$o sequence/amplicons.fa $f"
    echo $cm
    echo `date`
    $cm
    
done 

bwa mem -t 10 -o bam/ex3_1.bam sequence/amplicons.fa fastq_processed/umi_extract/ex3_1.assembled.fastq
Thu Mar 3 18:57:38 PST 2022
bwa mem -t 10 -o bam/ex3_2.bam sequence/amplicons.fa fastq_processed/umi_extract/ex3_2.assembled.fastq
Thu Mar 3 18:57:42 PST 2022
bwa mem -t 10 -o bam/ex4_1.bam sequence/amplicons.fa fastq_processed/umi_extract/ex4_1.assembled.fastq
Thu Mar 3 18:57:47 PST 2022
bwa mem -t 10 -o bam/ex4_2.bam sequence/amplicons.fa fastq_processed/umi_extract/ex4_2.assembled.fastq
Thu Mar 3 18:57:52 PST 2022
bwa mem -t 10 -o bam/NC1_ex3.bam sequence/amplicons.fa fastq_processed/umi_extract/NC1_ex3.assembled.fastq
Thu Mar 3 18:57:57 PST 2022
bwa mem -t 10 -o bam/NC1_ex4.bam sequence/amplicons.fa fastq_processed/umi_extract/NC1_ex4.assembled.fastq
Thu Mar 3 18:58:02 PST 2022
bwa mem -t 10 -o bam/NC2_ex3.bam sequence/amplicons.fa fastq_processed/umi_extract/NC2_ex3.assembled.fastq
Thu Mar 3 18:58:06 PST 2022
bwa mem -t 10 -o bam/NC2_ex4.bam sequence/amplicons.fa fastq_processed/

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 646461 sequences (90163978 bp)...
[M::mem_process_seqs] Processed 646461 reads in 28.912 CPU sec, 2.938 real sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa mem -t 10 -o bam/ex3_1.bam sequence/amplicons.fa fastq_processed/umi_extract/ex3_1.assembled.fastq
[main] Real time: 4.408 sec; CPU: 30.158 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 613290 sequences (85477075 bp)...
[M::mem_process_seqs] Processed 613290 reads in 27.171 CPU sec, 2.765 real sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa mem -t 10 -o bam/ex3_2.bam sequence/amplicons.fa fastq_processed/umi_extract/ex3_2.assembled.fastq
[main] Real time: 4.062 sec; CPU: 28.266 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 437494 sequences (100000266 bp)...
[M::process] read 256785 sequences (58707800 bp)...
[M::mem_process_seqs] Processed 437494 reads in 23.036 CPU sec, 2.477 real sec
[M::mem_process_seqs] P

In [13]:
!mv -v bam bam_amplicons

‘bam’ -> ‘bam_amplicons’


## 4. Remove dups

    samtools index example.bam
    
    ./umicollapse bam -i $bam -o $dd.bam --umi-sep _ --two-pass

https://github.com/Daniel-Liu-c0deb0t/UMICollapse

In [29]:
%%bash
wd=`pwd`
cd /data_gilbert/home/aarab/Workflows/UMICollapse/
for bam in $wd/bam/*srt.bam; do 
    dd_bam=${bam/.bam/.dd.bam};
    cm="./umicollapse bam -i $bam -o $dd_bam --umi-sep _ --two-pass"
    echo $bam
    echo `date`
    $cm
done
cd $wd

/data_gilbert/home/aarab/People/Benedict/bam/ex3_1.srt.bam
Fri Mar 4 12:03:40 PST 2022
Arguments	[bam, -i, /data_gilbert/home/aarab/People/Benedict/bam/ex3_1.srt.bam, -o, /data_gilbert/home/aarab/People/Benedict/bam/ex3_1.srt.dd.bam, --umi-sep, _, --two-pass]
Done with the first pass!
Number of input reads	646577
Number of removed unmapped reads	5245
Number of unremoved reads	641332
Number of unique alignment positions	165
Average number of UMIs per alignment position	2530.6666666666665
Max number of UMIs over all alignment positions	257093
Number of reads after deduplicating	60792
UMI collapsing finished in 8.841 seconds!
/data_gilbert/home/aarab/People/Benedict/bam/ex3_2.srt.bam
Fri Mar 4 12:03:49 PST 2022
Arguments	[bam, -i, /data_gilbert/home/aarab/People/Benedict/bam/ex3_2.srt.bam, -o, /data_gilbert/home/aarab/People/Benedict/bam/ex3_2.srt.dd.bam, --umi-sep, _, --two-pass]
Done with the first pass!
Number of input reads	613392
Number of removed unmapped reads	4365
Number of unremo

In [19]:
ls -lh bam

total 2.4G
-rw-rw-r--. 1 aarab aarab 255M Mar  4  2022 ex3_1.bam
-rw-rw-r--. 1 aarab aarab 3.3M Mar  4  2022 ex3_1.dd.bam
-rw-rw-r--. 1 aarab aarab 241M Mar  4  2022 ex3_2.bam
-rw-rw-r--. 1 aarab aarab 3.1M Mar  4  2022 ex3_2.dd.bam
-rw-rw-r--. 1 aarab aarab 388M Mar  4  2022 ex4_1.bam
-rw-rw-r--. 1 aarab aarab 5.2M Mar  4  2022 ex4_1.dd.bam
-rw-rw-r--. 1 aarab aarab 376M Mar  4  2022 ex4_2.bam
-rw-rw-r--. 1 aarab aarab 5.1M Mar  4  2022 ex4_2.dd.bam
-rw-rw-r--. 1 aarab aarab 268M Mar  4  2022 NC1_ex3.bam
-rw-rw-r--. 1 aarab aarab 3.6M Mar  4  2022 NC1_ex3.dd.bam
-rw-rw-r--. 1 aarab aarab 315M Mar  4  2022 NC1_ex4.bam
-rw-rw-r--. 1 aarab aarab 4.7M Mar  4  2022 NC1_ex4.dd.bam
-rw-rw-r--. 1 aarab aarab 262M Mar  4  2022 NC2_ex3.bam
-rw-rw-r--. 1 aarab aarab 3.4M Mar  4  2022 NC2_ex3.dd.bam
-rw-rw-r--. 1 aarab aarab 321M Mar  4  2022 NC2_ex4.bam
-rw-rw-r--. 1 aarab aarab 4.7M Mar  4  2022 NC2_ex4.dd.bam


In [30]:
%%bash 
for bam in bam/*.dd.bam; do
    cm="samtools index -@ 20 $bam"
    echo $cm
    echo `date`
    $cm
done 

samtools index -@ 20 bam/ex3_1.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/ex3_2.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/ex4_1.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/ex4_2.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/NC1_ex3.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/NC1_ex4.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/NC2_ex3.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022
samtools index -@ 20 bam/NC2_ex4.srt.dd.bam
Fri Mar 4 12:05:08 PST 2022


In [32]:
!zip bam.zip bam/*dd.bam*

  adding: bam/ex3_1.srt.dd.bam (deflated 0%)
  adding: bam/ex3_1.srt.dd.bam.bai (deflated 100%)
  adding: bam/ex3_2.srt.dd.bam (deflated 0%)
  adding: bam/ex3_2.srt.dd.bam.bai (deflated 100%)
  adding: bam/ex4_1.srt.dd.bam (deflated 0%)
  adding: bam/ex4_1.srt.dd.bam.bai (deflated 100%)
  adding: bam/ex4_2.srt.dd.bam (deflated 0%)
  adding: bam/ex4_2.srt.dd.bam.bai (deflated 100%)
  adding: bam/NC1_ex3.srt.dd.bam (deflated 0%)
  adding: bam/NC1_ex3.srt.dd.bam.bai (deflated 100%)
  adding: bam/NC1_ex4.srt.dd.bam (deflated 0%)
  adding: bam/NC1_ex4.srt.dd.bam.bai (deflated 100%)
  adding: bam/NC2_ex3.srt.dd.bam (deflated 0%)
  adding: bam/NC2_ex3.srt.dd.bam.bai (deflated 100%)
  adding: bam/NC2_ex4.srt.dd.bam (deflated 0%)
  adding: bam/NC2_ex4.srt.dd.bam.bai (deflated 100%)


## 5. find mutations
    
    samtools mpileup -f <ref> $bam > $pileup

In [208]:
ls sequence/amplicons.fa*

sequence/amplicons.fa      sequence/amplicons.fa.bwt  sequence/amplicons.fa.sa
sequence/amplicons.fa.amb  sequence/amplicons.fa.fai
sequence/amplicons.fa.ann  sequence/amplicons.fa.pac


http://seqanswers.com/forums/archive/index.php/t-30865.html

In [219]:
%%bash
for bam in bam/*.dd.bam; do 
    srt=${bam/.bam/.srt.bam};
    cm="samtools sort -@ 40 -o $srt $bam"
    echo $cm
    echo `date`
    $cm
done

samtools sort -@ 40 -o bam/ex3_1.dd.srt.bam bam/ex3_1.dd.bam
Thu Mar 3 19:45:16 PST 2022
samtools sort -@ 40 -o bam/ex3_2.dd.srt.bam bam/ex3_2.dd.bam
Thu Mar 3 19:45:16 PST 2022
samtools sort -@ 40 -o bam/ex4_1.dd.srt.bam bam/ex4_1.dd.bam
Thu Mar 3 19:45:16 PST 2022
samtools sort -@ 40 -o bam/ex4_2.dd.srt.bam bam/ex4_2.dd.bam
Thu Mar 3 19:45:16 PST 2022
samtools sort -@ 40 -o bam/NC1_ex3.dd.srt.bam bam/NC1_ex3.dd.bam
Thu Mar 3 19:45:16 PST 2022
samtools sort -@ 40 -o bam/NC1_ex4.dd.srt.bam bam/NC1_ex4.dd.bam
Thu Mar 3 19:45:16 PST 2022
samtools sort -@ 40 -o bam/NC2_ex3.dd.srt.bam bam/NC2_ex3.dd.bam
Thu Mar 3 19:45:17 PST 2022
samtools sort -@ 40 -o bam/NC2_ex4.dd.srt.bam bam/NC2_ex4.dd.bam
Thu Mar 3 19:45:17 PST 2022


[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...
[bam_sort_core] merging from 0 files and 40 in-memory blocks...


In [220]:
!samtools view -H bam/ex3_1.dd.bam

@HD	VN:1.6	SO:unsorted
@SQ	SN:Exon3_amplicon	LN:138
@SQ	SN:Exon4_amplicon	LN:229
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1198-dirty	CL:bwa mem -t 10 -o bam/ex3_1.bam sequence/amplicons.fa fastq_processed/umi_extract/ex3_1.assembled.fastq


In [221]:
!samtools view -H bam/ex3_1.dd.srt.bam

@HD	VN:1.6	SO:coordinate
@SQ	SN:Exon3_amplicon	LN:138
@SQ	SN:Exon4_amplicon	LN:229
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1198-dirty	CL:bwa mem -t 10 -o bam/ex3_1.bam sequence/amplicons.fa fastq_processed/umi_extract/ex3_1.assembled.fastq


In [223]:
%%bash
mkdir -p pileup
for bam in bam/*dd.srt.bam; do 
    base=`basename $bam`
    out=${base/.dd.srt.bam/.txt};
    cm="samtools mpileup -f sequence/amplicons.fa $bam -o pileup/$out"
    echo $cm
    echo `date`
    $cm
done

samtools mpileup -f sequence/amplicons.fa bam/ex3_1.dd.srt.bam -o pileup/ex3_1.txt
Thu Mar 3 19:45:57 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/ex3_2.dd.srt.bam -o pileup/ex3_2.txt
Thu Mar 3 19:46:00 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/ex4_1.dd.srt.bam -o pileup/ex4_1.txt
Thu Mar 3 19:46:02 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/ex4_2.dd.srt.bam -o pileup/ex4_2.txt
Thu Mar 3 19:46:06 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/NC1_ex3.dd.srt.bam -o pileup/NC1_ex3.txt
Thu Mar 3 19:46:09 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/NC1_ex4.dd.srt.bam -o pileup/NC1_ex4.txt
Thu Mar 3 19:46:12 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/NC2_ex3.dd.srt.bam -o pileup/NC2_ex3.txt
Thu Mar 3 19:46:15 PST 2022
samtools mpileup -f sequence/amplicons.fa bam/NC2_ex4.dd.srt.bam -o pileup/NC2_ex4.txt
Thu Mar 3 19:46:18 PST 2022


[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files


In [228]:
!zip -r pileup.zip pileup/

  adding: pileup/ (stored 0%)
  adding: pileup/ex3_1.txt (deflated 85%)
  adding: pileup/ex3_2.txt (deflated 87%)
  adding: pileup/ex4_1.txt (deflated 86%)
  adding: pileup/ex4_2.txt (deflated 86%)
  adding: pileup/NC1_ex3.txt (deflated 85%)
  adding: pileup/NC1_ex4.txt (deflated 85%)
  adding: pileup/NC2_ex3.txt (deflated 87%)
  adding: pileup/NC2_ex4.txt (deflated 86%)


In [3]:
cat sequence/amplicons.fa

>Exon3_amplicon
GATTCCTACAGGAAGCAAGTAGTAATTGATGGAGAAACCTGTCTCTTGGATATTCTCGACACAGCAGGTCAAGAGGAGTACAGTGCAATGAGGGACCAGTACATGAGGACTGGGGAGGGCTTTCTTTGTGTATTTGCC
>Exon4_amplicon
GTTGTGGACAGGTTTTGAAAGATATTTGTGTTACTAATGACTGTGCTATAACTTTTTTTTCTTTCCCAGAGAACAAATTAAAAGAGTTAAGGACTCTGAAGATGTACCTATGGTCCTAGTAGGAAATAAATGTGATTTGCCTTCTAGAACAGTAGACACAAAACAGGCTCAGGACTTAGCAAGAAGTTATGGAATTCCTTTTATTGAAACATCAGCAAAGACAAGACAG


In [2]:
!cat pileup/ex3_2.txt | awk '{print $1,$2,$3}' 

Exon3_amplicon 1 G
Exon3_amplicon 2 A
Exon3_amplicon 3 T
Exon3_amplicon 4 T
Exon3_amplicon 5 C
Exon3_amplicon 6 C
Exon3_amplicon 7 T
Exon3_amplicon 8 A
Exon3_amplicon 9 C
Exon3_amplicon 10 A
Exon3_amplicon 11 G
Exon3_amplicon 12 G
Exon3_amplicon 13 A
Exon3_amplicon 14 A
Exon3_amplicon 15 G
Exon3_amplicon 16 C
Exon3_amplicon 17 A
Exon3_amplicon 18 A
Exon3_amplicon 19 G
Exon3_amplicon 20 T
Exon3_amplicon 21 A
Exon3_amplicon 22 G
Exon3_amplicon 23 T
Exon3_amplicon 24 A
Exon3_amplicon 25 A
Exon3_amplicon 26 T
Exon3_amplicon 27 T
Exon3_amplicon 28 G
Exon3_amplicon 29 A
Exon3_amplicon 30 T
Exon3_amplicon 31 G
Exon3_amplicon 32 G
Exon3_amplicon 33 A
Exon3_amplicon 34 G
Exon3_amplicon 35 A
Exon3_amplicon 36 A
Exon3_amplicon 37 A
Exon3_amplicon 38 C
Exon3_amplicon 39 C
Exon3_amplicon 40 T
Exon3_amplicon 41 G
Exon3_amplicon 42 T
Exon3_amplicon 43 C
Exon3_amplicon 44 T
Exon3_amplicon 45 C
Exon3_amplicon 46 T
Exon3_amplicon 47 T
Exon3_amplicon 48 G
Exon3_amplicon 49 G
Exon3_amplicon 50 A
Exon3_amp

In [12]:
!date

Mon Feb 28 13:57:58 PST 2022
